<a href="https://colab.research.google.com/github/03axdov/Notebooks/blob/main/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.8.2


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [ ]:
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28, 1)
(60000,)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

In [ ]:
class Model(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10, activation="softmax")

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = Model()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape: # Keeps track of gradients
    predictions = model(images, training = True) # Training = True only necessary if there are layers that behave differently, e.g Dropout
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training = False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for images, labels in test_ds:
    test_step(images, labels)

  print(
      f"Epoch: {epoch + 1}, "
      f"Loss: {train_loss.result() * 100}, "
      f'Accuracy: {train_accuracy.result() * 100}, '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch: 1, Loss: 4.728362560272217, Accuracy: 98.51667022705078, Test Loss: 0.025017626583576202, Test Accuracy: 99.21333312988281
Epoch: 2, Loss: 2.590163230895996, Accuracy: 99.22166442871094, Test Loss: 0.01390406396239996, Test Accuracy: 99.5616683959961
Epoch: 3, Loss: 1.677040934562683, Accuracy: 99.41166687011719, Test Loss: 0.021389255300164223, Test Accuracy: 99.20832824707031
Epoch: 4, Loss: 1.0994330644607544, Accuracy: 99.59666442871094, Test Loss: 0.005073112901300192, Test Accuracy: 99.86333465576172
Epoch: 5, Loss: 0.7462408542633057, Accuracy: 99.7699966430664, Test Loss: 0.003767740912735462, Test Accuracy: 99.87999725341797
